# Entrega
## Grafos
En esta instancia, se obtiene un conjunto de datos de una plataforma de comida ya establecida. El mismo contiene los menues disponibles en la plataforma para distintos negocios de comida.

In [1]:
import pandas as pd
samples = pd.read_csv('datasets/comida_dataset.csv')
samples.head()

,section_name,category_name,category_tag_name,product_name,product_description
0,Promociones,Chivitos,Canadiense al plato,Chivito canadiense al plato para 2 con gramajo,"2 churrascos de lomo con jamón, muzza, panceta..."
1,Combinados de sushi,Sushi,sushi,Combinado Magnum - 30 piezas variadas,"Variedad de rolls, nigiris y sashimis."
2,Cucuruchos,Helados,cucurucho,Cucurucho natural,NaN
3,Sugerencia del día,Pescados y Mariscos,Merluza,Sugerencia - Merluza fresca con papas,NaN
4,Almuerzos por $99,Comida Árabe,Shawarma,Almuerzos por $99 - Shawarma Vegetariano,"Precio Anterior: $200, Ahora: $99."


Se puede visualizar una seccion nombre, donde se encuentra una descripcion del menu, la cual no aporta demasiada informacion. Luego estan nombre y tag-nombre de categoria, los cuales se acercan mas a la descripcion del menu que la columna anterior, pero que igualmente se decide descartar. Por ultimo, se tienen nombre y descripcion de producto, las columnas que mas utilidad aportan a la tarea que se plantea, pues contienen los ingredientes empleados.

Se hara uso de la ultima columna para obtener la informacion real de la composicion del producto ofrecido.

Se procede rellenando el conjunto de datos para evitar valores del tipo *NaN*. 

In [2]:
samples.fillna("", inplace = True)
samples.shape 

(99396, 5)

Dado que al negocio le interesa producir hamburguesas, se quiere informacion relacionada a eso, por lo que se decide filtrar en base a si se encuentra el termino o un sinonimo del mismo.

In [3]:
our_target_words = '"' + '|'.join(['hamburguesa', 'Hamburguesa', 'burger', 'Burger', 'sanguche', 'Sanguche', 'sándwich', 'Sándwich']) + '|\+"'
burguer_samples = samples[samples['product_name'].str.contains(our_target_words)]
burguer_samples.shape

(7173, 5)

Se propone obtener una lista de tokens de la descripcion del producto, eliminando palabras de menos de 3 letras, eliminando numeros y obteniendo el stem de la palabra. De esta forma palabras como tomate y tomates se identifica como el mismo ingrediente. Luego se obtendra el grafo asociado a columna, tomando como nodos los stem de los ingredientes.

In [10]:
import itertools
import networkx as nx
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('spanish')
stop_words = nltk.corpus.stopwords.words('spanish')

our_targeted_stems = ['hamburgues', 'burg', 'sanguch', 'sandwich']

def clean_stems(tokens):
    stems = []
    for token in word_tokenize(tokens):
        if len(token)>2 and not any(i.isdigit() for i in token) and token not in stop_words:
            stem = stemmer.stem(token)
            if stem not in our_targeted_stems and stem not in stems: stems.append(stem)
    return stems

burguer_samples['stems'] = burguer_samples.apply(lambda row: clean_stems(row['product_description']), axis=1)
burguer_samples.head()

/Users/leandro/Documents/facultad/analisis_datos/analisis/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,section_name,category_name,category_tag_name,product_name,product_description,stems
33,Hamburguesas vegetarianas,Comida Vegetariana,hamburguesa vegetariana,Hamburguesa vegetariana ciser,"Hamburguesa de garbanzos y vegetales, Lechuga,...","[garbanz, vegetal, lechug, tomat, ceboll, ques..."
66,Vegetarianas burger,Comida Vegetariana,Vegetariana,Godiee burger,"200 g de carne de lentejas al pan con rúcula, ...","[carn, lentej, pan, rucul, albahac, brot, soj,..."
91,Hamburguesas caseras,Comida Vegetariana,hamburguesa vegetariana,Hamburguesa vegetariana,"Doble hamburguesa de lentejas y verduras,cebol...","[dobl, lentej, verdur, ceboll, grill, lechug, ..."
181,Hamburguesas,Comida Vegetariana,hamburguesa vegetariana,Hamburguesa vegetariana,"Pan artesanal con sésamo, hamburguesa de espin...","[pan, artesanal, sesam, espinac, lechug, tomat..."
258,Los de siempre,Comida Vegetariana,hamburguesa vegetariana,Hamburguesa Vegetariana en pan batta,"Dos hamburguesas, con mayonesa, lechuga, tomat...","[dos, mayones, lechug, tomat, ceboll, acompañ,..."


#### Generamos un grafo donde los vértices están formados por ingredientes y hay una arista entre ellos si aparecen en el mismo menú.

In [14]:
def stem_graph(stems):
    G = nx.MultiGraph()
    for stem in stems:
        for (a, b) in list(itertools.combinations(stem, 2)):
            if not (a in our_targeted_stems or b in our_targeted_stems):
                G.add_edge(a, b) 
    return G

In [15]:
G = stem_graph(burguer_samples['stems'])

#### Inspeccionemos el grafo

In [21]:
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 937
Number of edges: 21685
Average degree:  46.2860


El mismo tiene 937 nodos y 21685 y en promedio, cada nodo tiene aproximadamente grado 46.

#### A continuación dibujamos la representación de los nodos con al menos grado 100 para poder observarlo mejor. A su vez, calculamos el Pagerank de cada uno y los representamos con un tamaño proporcional al mismo:

![graph](graph.png)

Se estudia la distribución de grado

![distribution](distribution.png)